## 크롤한 파일에 대한 스크래핑 프로세스

1. 스크래핑 프로세스
    - crawl 파일 폴더 읽기
    - 에러 체크
        * 에러시 : 에러파일 저장 폴더 이동, 재처리를 위한 체크 로직[처리 방식?]
        * 헤더를 읽어 해당 파싱 함수를 이용 하여 파싱 처리후 CSV 파일 저장[보관 기준?]
        * 파싱 완료 후 원본 데이터 이동[보관 기준?]
2. DB 프로세스
    - CSV 파일 폴더 읽기
    - 에러 체크
        * 에러시 : 에러파일 저장 폴더 이동, 재처리를 위한 체크 로직[처리 방식?]
        * csv 파일 읽어 DB 처리[테이블에 데이터 입력]
        * 처리 완료 파일 이동[보관 기준?]

### 기준 정보
- 폴더 정보
- 스크래핑 처리 포맷
    * 대상항공사코드,플라이트,출발일,출발지,도착지,출발시간,도착시간,가격,유류세,세금,좌석수

### 로그 처리
- log_util 의 common_logger 이용하여 로그 처리
-  log(messages,logger,level)

In [1]:
from collections import namedtuple
from datetime import datetime, timedelta
import requests
from common.env_variable import *
from common.batch_util import *
from common.scrap_func import *
from common.log_util import *
## 로그 초기화
logger_initialize('scrap_logger_setting.json')
init_env_variable('common/env_variable.json')

### 스크래핑 배치 파일

In [2]:
## 국내선 국제선 구분하여 폴더에 있는 파일 scraping 처리
## 포맷 : 대상항공사코드,플라이트,출발일,출발지,도착지,출발시간,도착시간,가격,유류세,세금,좌석수
## 오류 발생한 파일들은 해당 폴더에 그대로 방치, 주기적으로 체크하여 오래 남아 있는 파일들은 문제있는 파일로 취급 하여 재처리
def batch_scrap(crawl_dir,scrap_data_dir,dom_int=None):
    start_time = datetime.today()
    log_msgs = ['start batch job','scraping crawled file']
    log(log_msgs,level=logging.INFO)
    scrap_cnt = 0
    ## 스크랩 대상 파일(파일명, 내용)
    crawl_dir = crawl_dir
    files = get_crawled_file_list(crawl_dir)
    for file in files:
        log(file,level=logging.DEBUG)
        scrap_file = crawl_dir+'/'+file
        head, raw_data = read_crawled_file(scrap_file)
        ## 헤더 정보 읽어오기
        #head = get_headinfo(raw_data.splitlines()[0])
        log([head['dom_int'],head['site_code']],level=logging.DEBUG)
        if dom_int is not None and head['dom_int'] != dom_int: ## 국내선/국제선 체크
            continue
        site_code = head['site_code']
        ## 해당 함수 생성
        func = get_scrap_site_func(head['dom_int'],head['site_code'])
        log(['check func',func],level=logging.DEBUG)
        if func is None: ## 해당 함수가 없을 경우 다음으로
            continue
        scraped_list = func(head,raw_data)#''.join(raw_data.splitlines()[1:])))
        ## 스크래핑 실패시 해당 파일 폴더에 보관, 해당 폴더 모니터링시 장기 미처리 파일 체크하여 처리
        if scraped_list is None:
            ## 출렬 처리
            log('Error occured in {}!'.format(file),level=logging.DEBUG)
            continue
        ## CSV 처리
        scraped_list_to_csv(set_headinfo(**head),scraped_list,scrap_data_dir+"/"+file.split(".")[0]+".csv")
        ## 처리 파일 이동 처리
        ## 이동 처리시 이미 파일이 존재하는 경우 삭제후 처리
        if os.path.exists(os.path.join(SCRAP_OK_DIR,os.path.split(scrap_file)[-1])):
            os.remove(os.path.join(SCRAP_OK_DIR,os.path.split(scrap_file)[-1]))
        move_scraped_file(scrap_file,SCRAP_OK_DIR)
        scrap_cnt += 1

    end_time = datetime.today()
    log_msgs = ['end batch job','scraping crawled file',
                'elapsed -{}'.format(end_time-start_time),'Total crawl:{} files saved.'.format(scrap_cnt)]
    log(log_msgs,level=logging.INFO)

In [5]:
## 국내선, 국제선 스크래핑 처리
crawl_dir = CRAWL_DIR#'scrap_test'
scrap_data_dir = SCRAP_DATA_DIR
dom_int = None # 0 - 국내선, 1 - 국제선, None - 모두
batch_scrap(crawl_dir,scrap_data_dir,dom_int=dom_int)

2017-07-07 08:50:28,565 root     INFO     start batch job :: scraping crawled file
2017-07-07 08:50:28,570 root     DEBUG    scrap_2017070708_IP_7C_GMP_PUS_20170712.txt
2017-07-07 08:50:28,577 root     DEBUG    0 :: IP
2017-07-07 08:50:28,579 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-07-07 08:50:28,583 root     DEBUG    check func :: <function scraping_IP_dom at 0x000000000566F158>
2017-07-07 08:50:28,586 root     DEBUG    start scraping Interpark crawled data
2017-07-07 08:50:28,592 root     WARNING  scraping data not found!
2017-07-07 08:50:28,595 root     DEBUG    Error occured in scrap_2017070708_IP_7C_GMP_PUS_20170712.txt!
2017-07-07 08:50:28,597 root     DEBUG    scrap_2017070708_IP_7C_GMP_PUS_20170713.txt
2017-07-07 08:50:28,601 root     DEBUG    0 :: IP
2017-07-07 08:50:28,604 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-07-07 08:50:28,608 root     DEBUG    check func :: <function scraping_IP_dom at 0x000000000566F158>
2017-07-07 08:50:28,612 root     DEBUG    star

In [ ]:
## 크롤 데이터 테스트
with open('crawl/test.txt') as fp:
    json_data = json.load(fp)

In [ ]:
json_data['result'][0]['resultData'][0]